<a href="https://colab.research.google.com/github/bodamohannaik/DLAI-TF-DPC/blob/master/C3/W2/C3_W2_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds

# Dataset

In [2]:
help(tfds.load)

Help on function load in module tensorflow_datasets.core.load:

load(name: str, *, split: Union[tensorflow_datasets.core.splits.Split, Tuple[ForwardRef('Tree'), ...], List[ForwardRef('Tree')], Dict[str, ForwardRef('Tree')], NoneType] = None, data_dir: Union[str, NoneType] = None, batch_size: Union[int, NoneType] = None, shuffle_files: bool = False, download: bool = True, as_supervised: bool = False, decoders: Union[tensorflow_datasets.core.decode.base.Decoder, Dict[str, ForwardRef('TreeDict')], NoneType] = None, read_config: Union[tensorflow_datasets.core.utils.read_config.ReadConfig, NoneType] = None, with_info: bool = False, builder_kwargs: Union[Dict[str, Any], NoneType] = None, download_and_prepare_kwargs: Union[Dict[str, Any], NoneType] = None, as_dataset_kwargs: Union[Dict[str, Any], NoneType] = None, try_gcs: bool = False)
    Loads the named dataset into a `tf.data.Dataset`.
    
    `tfds.load` is a convenience method that:
    
    1. Fetch the `tfds.core.DatasetBuilder` by n

In [3]:
imdb, info =tfds.load(name="imdb_reviews", split=None, as_supervised=True, with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOUEQVT/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOUEQVT/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [5]:
imdb

{'test': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'train': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'unsupervised': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [6]:
imdb['train']

<PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [7]:
for text, label in imdb['train'].take(1):
  print(text.numpy())
  print(label.numpy())

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0


In [8]:
def separate_text_and_labels(data):
  sentences = []
  labels = []
  for text, label in data:
    sentences.append(text.numpy().decode())
    labels.append(label.numpy())
  labels = np.array(labels)
  return sentences, labels

In [9]:
train_sentences, train_labels = separate_text_and_labels(data=imdb['train'])
len(train_sentences), train_labels.shape

(25000, (25000,))

In [10]:
test_sentences, test_labels = separate_text_and_labels(data=imdb['test'])
len(test_sentences), test_labels.shape

(25000, (25000,))

# Text Tokenizer

In [11]:
# Text Tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, lower=True, char_level=False, split=' ', oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentences)

In [12]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)

In [13]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [14]:
maxlen = 120
padding = "post"
truncate = "post"

In [15]:
train_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences=train_sequences, maxlen=maxlen, padding = padding, truncating=truncate, value=0)

In [16]:
test_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences=test_sequences, maxlen=maxlen, padding = padding, truncating=truncate, value=0)

In [17]:
train_sequences.shape, test_sequences.shape

((25000, 120), (25000, 120))

# Neural Network classifier

In [18]:
model = tf.keras.Sequential(
    layers = [
              tf.keras.layers.Input(shape=(maxlen,)),
              tf.keras.layers.Embedding(input_dim=10000, output_dim=16),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dropout(rate = .2),
              tf.keras.layers.Dense(units=32, activation='relu'),
              tf.keras.layers.Dropout(rate = .2),
              tf.keras.layers.Dense(units=1, activation='sigmoid')
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dropout (Dropout)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 32)                61472     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 221,505
Trainable params: 221,505
Non-trai

In [19]:
# compile 
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics = ["accuracy"])

In [20]:
history = model.fit(x=train_sequences, y = train_labels, batch_size = 32, epochs = 10, validation_data = (test_sequences, test_labels))

Epoch 1/10
782/782 [==============================] - 8s 9ms/step - loss: 0.4896 - accuracy: 0.7445 - val_loss: 0.3840 - val_accuracy: 0.8271
Epoch 2/10
782/782 [==============================] - 7s 8ms/step - loss: 0.2536 - accuracy: 0.9004 - val_loss: 0.4438 - val_accuracy: 0.8134
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.1198 - accuracy: 0.9592 - val_loss: 0.5322 - val_accuracy: 0.8027
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.0492 - accuracy: 0.9847 - val_loss: 0.7163 - val_accuracy: 0.7964
Epoch 5/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0285 - accuracy: 0.9904 - val_loss: 0.8088 - val_accuracy: 0.8006
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0253 - accuracy: 0.9912 - val_loss: 0.8662 - val_accuracy: 0.7980
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0205 - accuracy: 0.9930 - val_loss: 0.9815 - val_accuracy: 0.7971
Epoch

In [21]:
embedding_layer = model.layers[0]

In [22]:
embedding_layer

In [25]:
embeddings = embedding_layer.weights
embeddings[0].shape

TensorShape([10000, 16])

In [28]:
# save embeddings 
vecs = open('vecs.tsv', 'w')
meta = open('meta.tsv', 'w')
for i in range(1, embeddings[0].shape[0]):
  meta.write(tokenizer.index_word[i]+"\n")
  vecs.write("\t".join([str(x.numpy()) for x in embeddings[0][i]])+"\n")
vecs.close()
meta.close()

In [27]:
from google.colab import files
files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
files.download('vecs.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>